<a href="https://colab.research.google.com/github/Parkgeunryeong/go/blob/master/minipro3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os, cv2, random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
drive.mount('/content/drive')

# 드라이브에서 데이터셋과 카테고리를 나누는 코드
dir_path = '/content/drive/MyDrive/dog_dataset'
categories = [
    'Beagle', 'Boxer', 'Bulldog', 'Dachshund', 'German_Shepherd', 'Golden_Retriever', 'Labrador_Retriever', 'Poodle', 'Rottweiler', 'Yorkshire_Terrier'
] # 클래스 수는 총 10개

# 이미지 로딩 및 전처리
data = []
for i in categories: #폴더 별로 데이터를 가져와서 폴더 명칭을 라벨로 예) Beagle은 0으로 라벨링...Boxer은 1로 라벨링...
    path = os.path.join(dir_path, i)
    label = categories.index(i)
    for img_name in os.listdir(path): # 각 폴더안에 들어있는 사진 데이터 전처리
        img_path = os.path.join(path, img_name)
        if os.path.splitext(img_path)[1].lower() == '.jpg': #.jpg파일만 가져오기
            img = cv2.imread(img_path)
            img = cv2.resize(img, (32, 32)) #이미지를 32,32사이즈로 변경
            data.append([img, label])

random.shuffle(data)

# 정규화 및 전처리
x = []
y = []
for features, label in data: #x에는 사진데이터 값, y는 라벨값
    x.append(features)
    y.append(label)

x = np.array(x) / 255.0 #0~255사이의 픽셀값을 0~1로 정규화
y = np.array(y)
x = x.reshape(-1, 32, 32, 3) #32,32로 다시 reshape

# 학습용 80%, 테스트용 20%로 분리
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, shuffle=True
)
#데이터 증가를 위한 부분
datagen = ImageDataGenerator(
    validation_split=0.2,  # 학습용 데이터중 20%를 검증셋에 이용
    rotation_range=30, #회전 30도까지
    width_shift_range=0.2,#좌우 이동
    zoom_range=[0.8, 1.2] #확대
)
#데이터 증가를 사용하기위한 제네레이터(학습데이터)
train_generator = datagen.flow(
    X_train, y_train,
    batch_size=20,
    subset='training'
)
#데이터 증가를 사용하기 위한 제네레이터(검증데이터)
val_generator = datagen.flow(
    X_train, y_train,
    batch_size=20,
    subset='validation'
)


# 모델 구성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))  # 10개의 카테고리 분류 (다중분류 이므로 softmax사용)
model.summary() #모델 구조 보여주는 함수

# 과적합 방지를 위한 early_stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=30)
checkpointer = ModelCheckpoint(filepath='/content/dog_model_best.keras', monitor='val_loss', save_best_only=True)

# 컴파일 및 학습
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, validation_data=val_generator, epochs=200, callbacks=[early_stopping, checkpointer])

# 평가
score = model.evaluate(X_test, y_test)
print("전체 데이터 정확도:", score[1])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 361,034 (1.38 MB)

 Trainable params: 361,034 (1.38 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.0869 - loss: 2.3190 - val_accuracy: 0.1429 - val_loss: 2.2930
Epoch 2/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.1110 - loss: 2.2885 - val_accuracy: 0.2143 - val_loss: 2.1996
Epoch 3/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.1917 - loss: 2.1817 - val_accuracy: 0.1688 - val_loss: 2.1686
Epoch 4/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.2255 - loss: 2.1112 - val_accuracy: 0.2013 - val_loss: 2.0566
Epoch 5/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.2723 - loss: 2.0172 - val_accuracy: 0.2208 - val_loss: 1.9714
Epoch 6/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.3388 - loss: 1.9193 - val_accuracy: 0.2532 - val_loss: 1.9020
Epoch 7/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.3429 - loss: 1.8620 - val_accuracy: 0.2987 - val_loss: 1.9200
Epoch 8/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.3864 - loss: 1.7466 - val_accuracy: 0.